In [1]:
import csv
data= list()
with open('redditSubmissions.csv', encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append(row)

In [2]:
# possible features
    # if it was very popular for that day  ( appeared in top of day section)
timeOrg = data[21343]['rawtime']
print(timeOrg)

import re
# time1 = re.sub('.*?T', '', timeOrg)
# time1 = re.sub(':..-07:00', '', time1)
# hour  = re.sub(':..', '', time1)
# minute = re.sub('..:', '', time1)

# minAsFracHour = int(minute)/60
# print(time)
# print(hour)
# print(minute)

# hourPlusMinFrac = int(hour) +minAsFracHour
# print(hourPlusMinFrac)
# print(hourMinSec)

import time
time1 = re.sub('......$', '', timeOrg)
timeBetter = time.strptime(time1, "%Y-%m-%dT%H:%M:%S")
print(timeBetter[3])


2012-09-16T12:54:41+00:00
12


In [26]:
#Sorted Total scores per hour of day
import time
import re
from collections import defaultdict
scorePerHour = defaultdict(int)
timeCheck = list()
for d in data:
    
    ratingMetric = d['score']
    dateTime = d['rawtime']
    timeCheck.append(dateTime)
    if(dateTime):
        ISOtime = re.sub('......$', '', dateTime)
        while(len(ISOtime)>19):
            ISOtime = re.sub('.$', '', ISOtime)
        timeStruct= time.strptime(ISOtime, "%Y-%m-%dT%H:%M:%S")
        hour = timeStruct[3]
        scorePerHour[hour]+= int(ratingMetric)
        
sortedScorePerHour = list(scorePerHour.items())
sortedScorePerHour.sort(key=lambda tup: tup[1])
sortedScorePerHour.reverse()
print(sortedScorePerHour)
print(timeStruct)


[(15, 1697185), (14, 1693439), (11, 1666427), (17, 1658390), (10, 1636416), (12, 1632518), (13, 1624336), (16, 1600815), (18, 1547066), (9, 1532880), (8, 1486000), (7, 1477584), (19, 1469200), (20, 1345067), (6, 1273782), (21, 1120452), (5, 1029631), (22, 981244), (23, 892566), (4, 805696), (0, 733965), (3, 669417), (2, 646559), (1, 623006)]
time.struct_time(tm_year=2012, tm_mon=9, tm_mday=15, tm_hour=14, tm_min=0, tm_sec=43, tm_wday=5, tm_yday=259, tm_isdst=-1)


In [27]:
#Sorted avg scores per hour of day
import time
import re
from collections import defaultdict
scorePerHour = defaultdict(int)
timeCounter = defaultdict(int)


def convertToHour(dateT):
    ISOtime = re.sub('......$', '', dateT)
    while(len(ISOtime)>19):
        ISOtime = re.sub('.$', '', ISOtime)
    timeStruct= time.strptime(ISOtime, "%Y-%m-%dT%H:%M:%S")
    return timeStruct[3]

for d in data:
    
    ratingMetric = d['score']
    dateTime = d['rawtime']
#     timeCheck.append(dateTime)
    if(dateTime):
        hour = convertToHour(dateTime)
        scorePerHour[hour]+= int(ratingMetric)
        timeCounter[hour]+=1
        

avgScore = list()
for (hour,tot) in scorePerHour.items():
    avgScore.append((hour,tot/timeCounter[hour]))
    
avgScore.sort(key=lambda tup: tup[1])
avgScore.reverse()

avgScoreDict = dict(avgScore)
print(avgScoreDict)

{7: 316.8741153763672, 8: 305.25883319638456, 9: 298.05172078553375, 6: 297.19598693420437, 10: 296.72094288304623, 11: 291.6903553299492, 12: 280.8874741913283, 13: 266.0229282672781, 5: 265.43722608919825, 14: 264.6411939365526, 15: 250.76610520094562, 17: 242.59654768870683, 16: 239.3205262371057, 18: 223.98523237295498, 4: 208.2439906952701, 19: 203.6877859420491, 3: 185.94916666666666, 20: 184.53381808204142, 2: 170.59604221635885, 23: 169.33523050654526, 21: 165.6738133964217, 22: 164.6935213158778, 0: 152.94123775786622, 1: 149.36609925677297}


In [30]:
import numpy as np
import scipy.optimize
import random
from collections import defaultdict
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm

dataWTime = list()
for d in data:
    if (d['rawtime']):
        if (d['score']):
             dataWTime.append(d)

def avgHourRating(ISOtime):
    feat =[1]
    hour =convertToHour(ISOtime)
    avgRating = int(avgScore[hour][1])
    feat.append(avgRating)
    return feat

        
X = [avgHourRating(d['rawtime']) for d in dataWTime]
y = [int(d['score']) for d in dataWTime]

print(X[0])

halfLenData = len(dataWTime)/2
# print(halfLenData)

X_train = X[:66153]
y_train = y[:66153]

X_test = X[66153:]
y_test = y[66153:]

print(y_test[2312])

theta1,residuals,rank,s = np.linalg.lstsq(X_train, y_train,rcond= -1)
predictionsTrn = [sum(x*theta1) for x in X_train]
MSELinRegTrn = np.square(np.subtract(y_train,predictionsTrn)).mean() 
predictionsTst = [sum(x*theta1) for x in X_test]

MSELinRegTst = np.square(np.subtract(y_test,predictionsTst)).mean() 
print("Lin reg for max avg score: ")
print("MSE for Training " +str(MSELinRegTrn))
print("MSE for Test "+str(MSELinRegTst))

[1, 239]
3
Lin reg for max avg score: 
MSE for Training 247922.07231436795
MSE for Test 215073.27789919128


In [36]:
import numpy as np
import scipy.optimize
import random
from collections import defaultdict
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm

import time
import re
from collections import defaultdict
scorePerHour = defaultdict(int)
timeCounter = defaultdict(int)


def convertToHour(dateT):
    ISOtime = re.sub('......$', '', dateT)
    while(len(ISOtime)>19):
        ISOtime = re.sub('.$', '', ISOtime)
    timeStruct= time.strptime(ISOtime, "%Y-%m-%dT%H:%M:%S")
    return timeStruct[3]

for d in data:
    
    ratingMetric = d['score']
    dateTime = d['rawtime']
#     timeCheck.append(dateTime)
    if(dateTime):
        hour = convertToHour(dateTime)
        scorePerHour[hour]+= int(ratingMetric)
        timeCounter[hour]+=1
        

avgScore = list()
for (hour,tot) in scorePerHour.items():
    avgScore.append((hour,tot/timeCounter[hour]))
    
avgScore.sort(key=lambda tup: tup[1])
avgScore.reverse()

avgScoreDict = dict(avgScore)
             
def avgHourRating(ISOtime):
    feat =[1]
    hour =convertToHour(ISOtime)
    totScores =dict(scorePerHour)
    totRating = int(totScores[hour])
    feat.append(totRating)
    return feat



X = [avgHourRating(d['rawtime']) for d in data]
y = [int(d['score']) for d in data]

print(X[0])

halfLenData = int(len(data)/2)
# print(halfLenData)

X_train = X[:halfLenData]
y_train = y[:halfLenData]

X_test = X[halfLenData:]
y_test = y[halfLenData:]



print(y_test[2312])
theta1,residuals,rank,s = np.linalg.lstsq(X_train, y_train,rcond= -1)
predictionsTrn = [sum(x*theta1) for x in X_train]
MSELinRegTrn = np.square(np.subtract(y_train,predictionsTrn)).mean() 
predictionsTst = [sum(x*theta1) for x in X_test]

MSELinRegTst = np.square(np.subtract(y_test,predictionsTst)).mean() 
print("Lin reg for max total score: ")
print("MSE for Training " +str(MSELinRegTrn))
print("MSE for Test "+str(MSELinRegTst))

[1, 1632518]
3
Lin reg for max total score: 
MSE for Training 246654.5366900134
MSE for Test 213823.71802280267


In [41]:

# for (user,images) in usersImageSubs.items():
#     for (user2,images2) in usersImageSubs.items():
#         if ((userSim[user][user2] == 0) & (user2 != user) & (len(images)>0) & (len(images2)>0)):
#             sameImgCount = 0
#             diffCount = 0
#             for a in images:
#                 if( a in images2):  
#                     sameImgCount+=1
#             sim = sameImgCount/(len(images)+len(images2)-sameImgCount)
#             userSim[user][user2] = sim
#             userSim[user2][user] = sim


# return score of most similar user / (LATER: 

('Animates_Everything', {'MickeyRooneyy': 0.09090909090909091, 'strikerthedj': 0.25, 'jiroyg': 0.5, 'Gangsta_Raper': 0.0011363636363636363, 'himdudemanguy': 0.25, 'gjgowey': 0.3333333333333333, 'CSmkII': 0.3333333333333333, 'speedfreek16': 0.5, 'STOPPP': 0.5, 'Duckman0121': 0.3333333333333333, 'kmp67': 0.03225806451612903, 'CrowKaneII': 0.25, 'azcomputerguru': 0.5, 'Moncole': 0.3333333333333333, 'shortguy014': 0.5, 'Pazzaz': 0.5, 'Hellothereawesome': 0.25, 'HadManySons': 0.25, 'koolkows': 0.5, 'IronOxide42': 0.3333333333333333, 'Killer2000': 0.5, 'TraumaticASH': 0.3333333333333333, 'MidgetDance1337': 0.5, 'LazyBurnbaby': 0.3333333333333333, 'todaysuckstomorrow': 0.5})
('Gangsta_Raper', {'classicalvin': 0.0005675368898978433, 'TheZeppelin1995': 0.0017006802721088435, 'cllphone09': 0.00170261066969353, 'xPowerTrain': 0.0011350737797956867, 'dja0794': 0.0005678591709256105, 'OmegaHunter9': 0.0005678591709256105, 'BlueScreen': 0.0005681818181818182, 'JustJoshWasTaken': 0.000568181818181818

In [73]:
from collections import defaultdict
import numpy as np
from collections import Counter
dataWimg = list()
for d in data:
    if (d['#image_id']):
        if (d['score']):
            dataWimg.append(d)
            
X = [(d['username'],d['#image_id']) for d in dataWimg]
y = [int(d['score']) for d in dataWimg]

halfLenData = len(dataWimg)/2
# print(halfLenData)

trnData = dataWimg [:66153]
X_train = X[:66153]
y_train = y[:66153]

X_test = X[66153:]
y_test = y[66153:]

usersImageSubs = defaultdict(set)
imageUserSubs = defaultdict(set)

for d in trnData:
    user = d['username']
    img = d['#image_id']
    usersImageSubs[user].add(img)
    imageUserSubs[img].add(user)
    
userUserDict = defaultdict(list) 
userSim = defaultdict(dict)    
for user in usersImageSubs:
    for img in usersImageSubs[user]:
        userUserDict[user]+= [u for u in imageUserSubs[img] if ((u !=user) & (u!=''))]
    userSim[user] = Counter(userUserDict[user])
        
def printFirst10(arr):
    counter = 0
    for a in arr:
        print(a)
        counter+=1
        if(counter>10):
            break

userJaccard = defaultdict(dict)
for user in userSim:
    if(user):
        for u2 in userSim[user]:
            if(u2):
                inters = userSim[user][u2]
                userJaccard[user][u2] = inters/(len(usersImageSubs[user])+len(usersImageSubs[u2])-inters)
    

# printFirst10(userJaccard.items())

scores = defaultdict(dict)
imageCount = defaultdict(int)
imageTotScore = defaultdict(int)
users = set()
images = set()
for d in trnData:
    scores[d['username']][d['#image_id']] = int(d['score'])
    users.add(d['username'])
    images.add(d['#image_id'])
    imageCount[d['#image_id']] +=1 
    imageTotScore[d['#image_id']] += int(d['total_votes'])

# if user not seen before, return img average
# Calculate img averages

avgImageScore = defaultdict(int)
for i in imageTotScore:
    avgImageScore[i] = imageTotScore[i]/imageCount[i]
    

# if img not seen before, return user average

# if none seen before, return overall average (for users with 1 post)

def weightedUserSimScore(user, img):
    score = 0
    if((user in users) & (img in images)):
        weightedSum = 0
        sumSim = 0
        for (u,v) in userJaccard[user].items():
            if(img in usersImageSubs[u]):
                weightedSum+= scores[u][img]*v
                sumSim+=v
        if(sumSim==0):
            sumSim = 1
        score = weightedSum/sumSim
    return score

predictionsTrn = [weightedUserSimScore(user,img) for (user,img) in X_train]

predictionsTst = [weightedUserSimScore(user,img) for (user,img) in X_test]


MSELinRegTrn = np.square(np.subtract(y_train,predictionsTrn)).mean() 
MSELinRegTst = np.square(np.subtract(y_test,predictionsTst)).mean() 
print("User similarity scores (without cold case baseline): ")
print("MSE for Training " +str(MSELinRegTrn))
print("MSE for Test "+str(MSELinRegTst))



User similarity scores (without cold case baseline): 
MSE for Training 325271.5599755894
MSE for Test 262443.68623212504


In [67]:
#user-user recc for rating
from collections import defaultdict
import numpy as np
from collections import Counter
dataWimg = list()
for d in data:
    if (d['#image_id']):
        if (d['number_of_upvotes']):
            if (d['total_votes']):
                dataWimg.append(d)
                
                
def returnRating(upvotes, totVotes):
    if(int(totVotes)==0):
        return 0
    return int(upvotes)/int(totVotes)
            
X = [(d['username'],d['#image_id']) for d in dataWimg]
y = [returnRating(d['number_of_upvotes'],d['total_votes']) for d in dataWimg]

halfLenData = len(dataWimg)/2
# print(halfLenData)

trnData = dataWimg [:66153]
X_train = X[:66153]
y_train = y[:66153]

X_test = X[66153:]
y_test = y[66153:]

usersImageSubs = defaultdict(set)
imageUserSubs = defaultdict(set)

for d in trnData:
    user = d['username']
    img = d['#image_id']
    usersImageSubs[user].add(img)
    imageUserSubs[img].add(user)
    
userUserDict = defaultdict(list) 
userSim = defaultdict(dict)    
for user in usersImageSubs:
    for img in usersImageSubs[user]:
        userUserDict[user]+= [u for u in imageUserSubs[img] if ((u !=user) & (u!=''))]
    userSim[user] = Counter(userUserDict[user])
        
def printFirst10(arr):
    counter = 0
    for a in arr:
        print(a)
        counter+=1
        if(counter>10):
            break

userJaccard = defaultdict(dict)
for user in userSim:
    if(user):
        for u2 in userSim[user]:
            if(u2):
                inters = userSim[user][u2]
                userJaccard[user][u2] = inters/(len(usersImageSubs[user])+len(usersImageSubs[u2])-inters)
    

# printFirst10(userJaccard.items())

ratings = defaultdict(dict)
imageCount = defaultdict(int)
imageTotRating = defaultdict(int)
users = set()
images = set()
for d in trnData:
    ratings[d['username']][d['#image_id']] = returnRating(d['number_of_upvotes'],d['total_votes'])
    users.add(d['username'])
    images.add(d['#image_id'])
    imageCount[d['#image_id']] +=1 
    imageTotRating[d['#image_id']] += returnRating(d['number_of_upvotes'],d['total_votes'])

# if user not seen before, return img average
# Calculate img averages

avgImageRating = defaultdict(int)
for i in imageTotRating:
    avgImageRating[i] = imageTotRating[i]/imageCount[i]
    

# if img not seen before, return user average

# if none seen before, return overall average (for users with 1 post)

def weightedUserSimScore(user, img):
    rating = 0
    if((user in users) & (img in images)):
        weightedSum = 0
        sumSim = 0
        for (u,v) in userJaccard[user].items():
            if(img in usersImageSubs[u]):
                weightedSum+= ratings[u][img]*v
                sumSim+=v
        if(sumSim==0):
            sumSim = 1
        rating = weightedSum/sumSim
    return rating

predictionsTrn = [weightedUserSimScore(user,img) for (user,img) in X_train]

predictionsTst = [weightedUserSimScore(user,img) for (user,img) in X_test]

# print("trn data pred")
# printFirst10(predictionsTrn)
# print("trn data actual")
# printFirst10(y_train)
# print("test data pred")
# printFirst10(predictionsTst)
# print("test data actual")
# printFirst10(y_test)

MSELinRegTrn = np.square(np.subtract(y_train,predictionsTrn)).mean() 
MSELinRegTst = np.square(np.subtract(y_test,predictionsTst)).mean() 
print("User-similarity Ratings (without cold case avg baseline): ")
print("MSE for Training " +str(MSELinRegTrn))
print("MSE for Test "+str(MSELinRegTst))

User-similarity Ratings (without cold case avg baseline): 
MSE for Training 0.07410284432886191
MSE for Test 0.41651326279032924


In [7]:
#user-user recc for rating with cold case baselines
from collections import defaultdict
import numpy as np
from collections import Counter
dataWimg = list()
for d in data:
    if (d['#image_id']):
        if (d['number_of_upvotes']):
            if (d['total_votes']):
                dataWimg.append(d)
                
                
def returnRating(upvotes, totVotes):
    if(int(totVotes)==0):
        return 0
    return int(upvotes)/int(totVotes)
            
X = [(d['username'],d['#image_id']) for d in dataWimg]
y = [returnRating(d['number_of_upvotes'],d['total_votes']) for d in dataWimg]


halfLenData = len(dataWimg)/2
# print(halfLenData)

trnData = dataWimg [:66153]
X_train = X[:66153]
y_train = y[:66153]

X_test = X[66153:]
y_test = y[66153:]

usersImageSubs = defaultdict(set)
imageUserSubs = defaultdict(set)

for d in trnData:
    user = d['username']
    img = d['#image_id']
    usersImageSubs[user].add(img)
    imageUserSubs[img].add(user)
    
userUserDict = defaultdict(list) 
userSim = defaultdict(dict)    
for user in usersImageSubs:
    for img in usersImageSubs[user]:
        userUserDict[user]+= [u for u in imageUserSubs[img] if ((u !=user) & (u!=''))]
    userSim[user] = Counter(userUserDict[user])
        
def printFirst10(arr):
    counter = 0
    for a in arr:
        print(a)
        counter+=1
        if(counter>10):
            break

userJaccard = defaultdict(dict)
for user in userSim:
    if(user):
        for u2 in userSim[user]:
            if(u2):
                inters = userSim[user][u2]
                userJaccard[user][u2] = inters/(len(usersImageSubs[user])+len(usersImageSubs[u2])-inters)
    

# printFirst10(userJaccard.items())

ratings = defaultdict(dict)
userCount = defaultdict(int)
userTotRating = defaultdict(int)
imageCount = defaultdict(int)
imageTotRating = defaultdict(int)
totRating =0
users = set()
images = set()
for d in trnData:
    ratings[d['username']][d['#image_id']] = returnRating(d['number_of_upvotes'],d['total_votes'])
    users.add(d['username'])
    images.add(d['#image_id'])
    imageCount[d['#image_id']] +=1 
    imageTotRating[d['#image_id']] += returnRating(d['number_of_upvotes'],d['total_votes'])
    userCount[d['username']] +=1 
    userTotRating[d['username']] += returnRating(d['number_of_upvotes'],d['total_votes'])
    totRating+=returnRating(d['number_of_upvotes'],d['total_votes'])
    
avgRating = totRating/len(trnData)
    
    # if user not seen before, return img average 
# Calculate img averages

avgImageRating = defaultdict(int)
for i in imageTotRating:
    avgImageRating[i] = imageTotRating[i]/imageCount[i]
    
avgUserRating = defaultdict(int)
for i in userTotRating:
    avgUserRating[i] = userTotRating[i]/userCount[i]
    

# if img not seen before, return user average

# if none seen before, return overall average (for users with 1 post)

def weightedUserSimScore(user, img):
    rating = 0
    if((user in users) & (img in images)):
        weightedSum = 0
        sumSim = 0
        for (u,v) in userJaccard[user].items():
            if(img in usersImageSubs[u]):
                weightedSum+= ratings[u][img]*v
                sumSim+=v
        if(sumSim==0):
            sumSim = 1
        return (weightedSum/sumSim)
    elif (img in images):
        return avgImageRating[img]
    elif (user in users):
        return avgUserRating[user]
    else:
        return avgRating

predictionsTrn = [weightedUserSimScore(user,img) for (user,img) in X_train]

predictionsTst = [weightedUserSimScore(user,img) for (user,img) in X_test]

# print("trn data pred")
# printFirst10(predictionsTrn)
# print("trn data actual")
# printFirst10(y_train)
# print("test data pred")
# printFirst10(predictionsTst)
# print("test data actual")
# printFirst10(y_test)

MSELinRegTrn = np.square(np.subtract(y_train,predictionsTrn)).mean() 
MSELinRegTst = np.square(np.subtract(y_test,predictionsTst)).mean() 
print("User similarity Ratings (with cold case avg baseline): ")
print("MSE for Training " +str(MSELinRegTrn))
print("MSE for Test "+str(MSELinRegTst))

printFirst

User similarity Ratings (with cold case avg baseline): 
MSE for Training 0.07410284432886191
MSE for Test 0.028245667746484827


In [64]:
#user-user recc for rating with cold case baselines and return avg for 1 user-resub
from collections import defaultdict
import numpy as np
from collections import Counter
dataWimg = list()
for d in data:
    if (d['#image_id']):
        if (d['number_of_upvotes']):
            if (d['total_votes']):
                dataWimg.append(d)
                
                
def returnRating(upvotes, totVotes):
    if(int(totVotes)==0):
        return 0
    return int(upvotes)/int(totVotes)
            
X = [(d['username'],d['#image_id']) for d in dataWimg]
y = [returnRating(d['number_of_upvotes'],d['total_votes']) for d in dataWimg]


halfLenData = len(dataWimg)/2
# print(halfLenData)

trnData = dataWimg [:66153]
X_train = X[:66153]
y_train = y[:66153]

X_test = X[66153:]
y_test = y[66153:]

usersImageSubs = defaultdict(set)
imageUserSubs = defaultdict(set)

for d in trnData:
    user = d['username']
    img = d['#image_id']
    usersImageSubs[user].add(img)
    imageUserSubs[img].add(user)
    
userUserDict = defaultdict(list) 
userSim = defaultdict(dict)    
for user in usersImageSubs:
    for img in usersImageSubs[user]:
        userUserDict[user]+= [u for u in imageUserSubs[img] if ((u !=user) & (u!=''))]
    userSim[user] = Counter(userUserDict[user])
        
def printFirst10(arr):
    counter = 0
    for a in arr:
        print(a)
        counter+=1
        if(counter>10):
            break

userJaccard = defaultdict(dict)
for user in userSim:
    if(user):
        for u2 in userSim[user]:
            if(u2):
                inters = userSim[user][u2]
                userJaccard[user][u2] = inters/(len(usersImageSubs[user])+len(usersImageSubs[u2])-inters)
    

# printFirst10(userJaccard.items())

ratings = defaultdict(dict)
userCount = defaultdict(int)
userTotRating = defaultdict(int)
imageCount = defaultdict(int)
imageTotRating = defaultdict(int)
totRating =0
users = set()
images = set()
for d in trnData:
    ratings[d['username']][d['#image_id']] = returnRating(d['number_of_upvotes'],d['total_votes'])
    users.add(d['username'])
    images.add(d['#image_id'])
    imageCount[d['#image_id']] +=1 
    imageTotRating[d['#image_id']] += returnRating(d['number_of_upvotes'],d['total_votes'])
    userCount[d['username']] +=1 
    userTotRating[d['username']] += returnRating(d['number_of_upvotes'],d['total_votes'])
    totRating+=returnRating(d['number_of_upvotes'],d['total_votes'])
    
avgRating = totRating/len(trnData)
    
    # if user not seen before, return img average 
# Calculate img averages

avgImageRating = defaultdict(int)
for i in imageTotRating:
    avgImageRating[i] = imageTotRating[i]/imageCount[i]
    
avgUserRating = defaultdict(int)
for i in userTotRating:
    avgUserRating[i] = userTotRating[i]/userCount[i]
    

# if img not seen before, return user average

# if none seen before, return overall average (for users with 1 post)

def weightedUserSimScore(user, img):
    rating = 0
    if((user in users) & (img in images)):
        weightedSum = 0
        sumSim = 0
        for (u,v) in userJaccard[user].items():
            if(img in usersImageSubs[u]):
                weightedSum+= ratings[u][img]*v
                sumSim+=v
        if(sumSim==0):
            sumSim = 1
        return (weightedSum/sumSim)
    elif (img in images):
        return avgImageRating[img]
    elif (user in users):
        return avgUserRating[user]
    else:
        return avgRating

predictionsTrn = [weightedUserSimScore(user,img) for (user,img) in X_train]

predictionsTst = [weightedUserSimScore(user,img) for (user,img) in X_test]

# print("trn data pred")
# printFirst10(predictionsTrn)
# print("trn data actual")
# printFirst10(y_train)
# print("test data pred")
# printFirst10(predictionsTst)
# print("test data actual")
# printFirst10(y_test)

MSELinRegTrn = np.square(np.subtract(y_train,predictionsTrn)).mean() 
MSELinRegTst = np.square(np.subtract(y_test,predictionsTst)).mean() 
print("User similarity Ratings (with cold case avg baseline): ")
print("MSE for Training " +str(MSELinRegTrn))
print("MSE for Test "+str(MSELinRegTst))

User similarity Ratings (with cold case avg baseline): 
MSE for Training 0.07410284432886191
MSE for Test 0.028245667746484827


In [76]:
counter = 0
ratingTot =0
for d in trnData:
    ratingTot += returnRating(d['number_of_upvotes'],d['total_votes'])
    counter+=1
ratingAvg = ratingTot/counter

predictionsTrn = [ratingAvg for (user,img) in X_train]
predictionsTst = [ratingAvg for (user,img) in X_test]

MSELinRegTrn = np.square(np.subtract(y_train,predictionsTrn)).mean() 
MSELinRegTst = np.square(np.subtract(y_test,predictionsTst)).mean() 
print("Simple average baseline (predict average) ")
print("MSE for Training " +str(MSELinRegTrn))
print("MSE for Test "+str(MSELinRegTst))

printFirst10(predictionsTrn)
printFirst10(predictionsTst)

Simple average baseline (predict average) 
MSE for Training 0.026373207593078332
MSE for Test 0.028009249573915963
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302
0.6307427733591302


In [69]:
# printFirst10(userJaccard.items())
printFirst10(predictionsTrn)


0.5226088112947311
0.5230103341791678
0.5230103341791678
0.5230103341791678
0.5230103341791678
0.5293037798060138
0.0
0.0
0.0
0.5272206450934578
0.5245163561532981


In [62]:
# Cosine similarity!
# user-user recc for rating with cold case baselines and return avg for 1 user-resub
from collections import defaultdict
import math
import numpy as np
from collections import Counter
dataWimg = list()
for d in data:
    if (d['#image_id']):
        if (d['number_of_upvotes']):
            if (d['total_votes']):
                dataWimg.append(d)
                
def returnRating(upvotes, totVotes):
    if(int(totVotes)==0):
        return 0
    return int(upvotes)/int(totVotes)
            
X = [(d['username'],d['#image_id']) for d in dataWimg]
y = [returnRating(d['number_of_upvotes'],d['total_votes']) for d in dataWimg]


halfLenData = len(dataWimg)/2
# print(halfLenData)

trnData = dataWimg [:66153]
X_train = X[:66153]
y_train = y[:66153]

X_test = X[66153:]
y_test = y[66153:]

usersImageSubs = defaultdict(list)
imageUserSubs = defaultdict(list)

for d in trnData:
    user = d['username']
    img = d['#image_id']
    usersImageSubs[user].append(img)
    imageUserSubs[img].append(user)
    
userUserDict = defaultdict(list) 
userSim = defaultdict(dict)    
for user in usersImageSubs:
    for img in usersImageSubs[user]:
        userUserDict[user]+= [u for u in imageUserSubs[img] if ((u !=user) & (u!=''))]
    userSim[user] = Counter(userUserDict[user])
        
def printFirst10(arr):
    counter = 0
    for a in arr:
        print(a)
        counter+=1
        if(counter>10):
            break
    

# printFirst10(userJaccard.items())

ratings = defaultdict(dict)
userCount = defaultdict(int)
userTotRating = defaultdict(int)
imageCount = defaultdict(int)
imageTotRating = defaultdict(int)
totRating =0
users = set()
images = set()
for d in trnData:
    ratings[d['username']][d['#image_id']] = returnRating(d['number_of_upvotes'],d['total_votes'])
    users.add(d['username'])
    images.add(d['#image_id'])
    imageCount[d['#image_id']] +=1 
    imageTotRating[d['#image_id']] += returnRating(d['number_of_upvotes'],d['total_votes'])
    userCount[d['username']] +=1 
    userTotRating[d['username']] += returnRating(d['number_of_upvotes'],d['total_votes'])
    totRating+=returnRating(d['number_of_upvotes'],d['total_votes'])
    
avgRating = totRating/len(trnData)
    
counterrr= 0

avgImageRating = defaultdict(int)
for i in imageTotRating:
    avgImageRating[i] = imageTotRating[i]/imageCount[i]
    
avgUserRating = defaultdict(int)
for i in userTotRating:
    avgUserRating[i] = userTotRating[i]/userCount[i]
    
    
userCosine = defaultdict(dict)
for user in userSim:
    if(user):
        for u2 in userSim[user]:
            dotProd = 0
            if(u2):
                for i in usersImageSubs[user]:
                    if (i in usersImageSubs[u2]):
                        dotProd+= ratings[user][i]*ratings[u2][i] 
                userCosine[user][u2] = dotProd/(math.sqrt(len(usersImageSubs[user]))*math.sqrt(len(usersImageSubs[u2])))
                
    
def weightedUserSimScore(user, img):
    rating = 0
    if((user in users) & (img in images)):
        weightedSum = 0
        sumSim = 0
        for (u,v) in userCosine[user].items():
            if(img in usersImageSubs[u]):
                weightedSum+= ratings[u][img]*v
                sumSim+=v
        if(sumSim==0):
            sumSim = 1
        return (weightedSum/sumSim)
    elif (img in images):
        return avgImageRating[img]
    elif (user in users):
        return avgUserRating[user]
    else:
        return avgRating

predictionsTrn = [weightedUserSimScore(user,img) for (user,img) in X_train]
predictionsTst = [weightedUserSimScore(user,img) for (user,img) in X_test]

# print("trn data pred")
# printFirst10(predictionsTrn)
# print("trn data actual")
# printFirst10(y_train)
# print("test data pred")
# printFirst10(predictionsTst)
# print("test data actual")
# printFirst10(y_test)

MSELinRegTrn = np.square(np.subtract(y_train,predictionsTrn)).mean() 
MSELinRegTst = np.square(np.subtract(y_test,predictionsTst)).mean() 
print("User similarity Ratings (with cold case avg baseline): ")
print("MSE for Training " +str(MSELinRegTrn))
print("MSE for Test "+str(MSELinRegTst))

User similarity Ratings (with cold case avg baseline): 
MSE for Training 0.07363918536066838
MSE for Test 0.028245667746484827


In [63]:
# printFirst10(userCosine.items())
# printFirst10(imageUserSubs.items())
# printFirst10(usersImageSubs.items())

# printFirst10(userCosine.items())
# for u2 in userSim['Animates_Everything']:
# for i in imageUserSubs['Animates_Everything']:
#     print(i)
# print(imageUserSubs['Animates_Everything'])
# print(counterrr)
# Animates_Everything


('Animates_Everything', {'Gangsta_Raper': 0.012107000723464179, 'Hellothereawesome': 0.1546257147489185, 'HadManySons': 0.16372134502826668, 'TraumaticASH': 0.12250632094652887, 'MidgetDance1337': 0.2819309754484297, 'Pazzaz': 0.1975467240327462, 'koolkows': 0.13860008044720987, 'Moncole': 0.1500702431594979, 'shortguy014': 0.13860008044720987, 'Killer2000': 0.1559250905031111, 'todaysuckstomorrow': 0.18191260558696296, 'LazyBurnbaby': 0.14942836872340082, 'azcomputerguru': 0.25337827206755553, 'IronOxide42': 0.2338876357546667, 'CrowKaneII': 0.17325010055901233, 'jiroyg': 0.29164388175039613, 'himdudemanguy': 0.24887879332065055, 'STOPPP': 0.1988481011934519, 'MickeyRooneyy': 0.11645502222672068, 'kmp67': 0.05882724306721172, 'Duckman0121': 0.16888982599431812, 'CSmkII': 0.17359754866095006, 'gjgowey': 0.09022272283380679, 'strikerthedj': 0.1323731068604115, 'speedfreek16': 0.15623779379485506})
('Gangsta_Raper', {'Animates_Everything': 0.05470052279816411, 'Hellothereawesome': 0.0366

In [20]:
l1 = [1,2,3,5]
l2 = [4,3,2,5]
l3 = [7,3,9,2]

l4 = list()
l4.append(l1)
l4.append(l2)
l4.append(l3)
l5 =list()
for a in l4:
    l5 +=l1
print(l5)

[1, 2, 3, 5, 1, 2, 3, 5, 1, 2, 3, 5]


In [88]:
# userSim = defaultdict(lambda: defaultdict(int))

# for (user,images) in usersSampleDict.items():
#     for (user2,images2) in usersSampleDict.items():
#         if ((userSim[user][user2] == 0) & (user2 != user)):
#             sameImgCount = 0
#             for a in images:
#                 if( a in images2):  
#                     sameImgCount+=1
#             sim = sameImgCount/(len(images)+len(images2)-sameImgCount)
#             userSim[user][user2] = sim
#             userSim[user2][user] = sim

KeyboardInterrupt: 

In [87]:
import math
def jaccardSim(a,b):
    sameImg = a.intersection(b)
    jaccardSimil = sameImg/(len(a)+len(b)-sameImg)
    return jaccardSimil

In [30]:
printFirst10(imageUserSubs.items())

('0', ['Animates_Everything', 'Gangsta_Raper', 'Gangsta_Raper', 'Gangsta_Raper', 'Gangsta_Raper', 'Hellothereawesome', '', '', '', 'HadManySons', 'TraumaticASH', 'MidgetDance1337', 'Pazzaz', '', 'koolkows', 'Moncole', 'shortguy014', 'Killer2000', 'todaysuckstomorrow', '', 'LazyBurnbaby', '', 'azcomputerguru', 'IronOxide42', '', 'Gangsta_Raper', 'CrowKaneII'])
('1', ['Hysteriia', 'IronicIvan', 'CeruleanThunder', 'The_Bhuda_Palm', 'LOUD_DUCK'])
('100', ['Sarbanes_Foxy', 'awcreative', 'potato1', 'watcher_of_the_skies', 'tianan', '', 'Spam4119', '', '', 'beardface909', 'Rockytriton', '', ''])
('1000', ['MrBlockOfCheese', 'Athejew', '', '', ''])
('10001', ['RIAEvangelist', 'herpichj', 'JGDC', 'charles1er', 'imaznumkay', 'AMightySandwich'])
('10002', ['PDog8481', 'ahhdum', 'deathofpatriotism', 'sedgewakeful', 'sfsam415'])
('10003', ['irishjack777', 'Scopolamina', 'AtheismFTW', 'Emwat1024', 'Gangsta_Raper'])
('10004', ['fitted', '', 'TistheSEASON', 'JackieBronassis'])
('10005', ['Moridin87', 

In [83]:
#Sorted avg scores per hour of day
import time
import re
from collections import defaultdict
scorePerMonth = defaultdict(int)
timeCounter = defaultdict(int)


def convertToMonth(dateT):
    ISOtime = re.sub('......$', '', dateT)
    while(len(ISOtime)>19):
        ISOtime = re.sub('.$', '', ISOtime)
    timeStruct= time.strptime(ISOtime, "%Y-%m-%dT%H:%M:%S")
    return timeStruct[1]
                
for d in data:
    
    ratingMetric = d['score']
    dateTime = d['rawtime']
#     timeCheck.append(dateTime)
    if(dateTime):
        month = convertToMonth(dateTime)
        scorePerMonth[month]+= int(ratingMetric)
        timeCounter[month]+=1
        
avgScoreM = list()
for (month,tot) in scorePerMonth.items():
    avgScoreM.append((month,tot/timeCounter[month]))
    
avgScoreM.sort(key=lambda tup: tup[1])
avgScoreM.reverse()

avgScoreMDict = dict(avgScoreM)
print(avgScoreMDict)

{2: 344.57908590571907, 3: 340.95228519195615, 5: 299.94111493050366, 4: 294.3210465452443, 1: 287.69272727272727, 6: 281.92391042619795, 12: 248.66270193502575, 11: 230.1021449576198, 7: 216.54105604241843, 8: 213.2427796695674, 10: 192.5027616680475, 9: 181.87694395966648}


In [85]:
import numpy as np
import scipy.optimize
import random
from collections import defaultdict
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm

dataWTime = list()
for d in data:
    if (d['rawtime']):
        if (d['score']):
             dataWTime.append(d)
                
def avgMonthRating(ISOtime):
    feat =[1]
    month =convertToMonth(ISOtime)
    avgRating = int(avgScoreMDict[month])
    feat.append(avgRating)
    return feat

X = [avgMonthRating(d['rawtime']) for d in dataWTime]
y = [int(d['score']) for d in dataWTime]

print(X[0])

halfLenData = len(dataWTime)/2
# print(halfLenData)

X_train = X[:66153]
y_train = y[:66153]

X_test = X[66153:]
y_test = y[66153:]

print(y_test[2312])

theta1,residuals,rank,s = np.linalg.lstsq(X_train, y_train,rcond= -1)
predictionsTrn = [sum(x*theta1) for x in X_train]
MSELinRegTrn = np.square(np.subtract(y_train,predictionsTrn)).mean() 
predictionsTst = [sum(x*theta1) for x in X_test]

MSELinRegTst = np.square(np.subtract(y_test,predictionsTst)).mean() 
print("Lin reg for avg total score Month: ")
print("MSE for Training " +str(MSELinRegTrn))
print("MSE for Test "+str(MSELinRegTst))
        

[1, 340]
3
Lin reg for avg total score Month: 
MSE for Training 245588.00790515443
MSE for Test 212777.12427167763


In [86]:
def printFirst10(arr):
    counter = 0
    for a in arr:
        print(a)
        counter+=1
        if(counter>10):
            break
            
printFirst10(predictionsTst)
printFirst10(y_test)

355.95276255257016
308.82515550451524
313.94772148799944
313.94772148799944
295.5064839474562
295.5064839474562
295.5064839474562
295.5064839474562
295.5064839474562
295.5064839474562
228.9131261621612
8
10
504
18
62
-8
-6
4
2
1
-2


In [65]:
print(data[0])

OrderedDict([('#image_id', '0'), ('unixtime', '1333172439'), ('rawtime', '2012-03-31T12:40:39.590113-07:00'), ('title', "And here's a downvote."), ('total_votes', '63470'), ('reddit_id', 'rmqjs'), ('number_of_upvotes', '32657'), ('subreddit', 'funny'), ('number_of_downvotes', '30813'), ('localtime', '1333197639'), ('score', '1844'), ('number_of_comments', '622'), ('username', 'Animates_Everything')])


In [60]:
usersImageSubs = defaultdict(list)
imageUserSubs = defaultdict(list)

for d in trnData:
    user = d['username']
    img = d['#image_id']
    usersImageSubs[user].append(img)
    imageUserSubs[img].append(user)
    
printFirst10(imageUserSubs.items())

('0', ['Animates_Everything', 'Gangsta_Raper', 'Gangsta_Raper', 'Gangsta_Raper', 'Gangsta_Raper', 'Hellothereawesome', '', '', '', 'HadManySons', 'TraumaticASH', 'MidgetDance1337', 'Pazzaz', '', 'koolkows', 'Moncole', 'shortguy014', 'Killer2000', 'todaysuckstomorrow', '', 'LazyBurnbaby', '', 'azcomputerguru', 'IronOxide42', '', 'Gangsta_Raper', 'CrowKaneII'])
('1', ['Hysteriia', 'IronicIvan', 'CeruleanThunder', 'The_Bhuda_Palm', 'LOUD_DUCK'])
('100', ['Sarbanes_Foxy', 'awcreative', 'potato1', 'watcher_of_the_skies', 'tianan', '', 'Spam4119', '', '', 'beardface909', 'Rockytriton', '', ''])
('1000', ['MrBlockOfCheese', 'Athejew', '', '', ''])
('10001', ['RIAEvangelist', 'herpichj', 'JGDC', 'charles1er', 'imaznumkay', 'AMightySandwich'])
('10002', ['PDog8481', 'ahhdum', 'deathofpatriotism', 'sedgewakeful', 'sfsam415'])
('10003', ['irishjack777', 'Scopolamina', 'AtheismFTW', 'Emwat1024', 'Gangsta_Raper'])
('10004', ['fitted', '', 'TistheSEASON', 'JackieBronassis'])
('10005', ['Moridin87', 